In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('window-size=1920x1080')
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options)

In [2]:
import time

driver.get('https://www.abf.gov.au/help-and-support/notices/australian-customs-notices')
time.sleep(2)

In [3]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [4]:
temp = soup.find('ul', attrs={'class' : 'pagination'}).find_all("li")
# len(temp)

In [5]:
from selenium.webdriver.common.by import By
import re

In [6]:
dates = []
titles = []
links = []
last_page = len(temp)

for i in range(2,last_page):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    date_rows = soup.select('div.col-sm-3 span')
    title_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-header')})
    link_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-content')})
    
    for row in date_rows:
        date = row.text
        dates.append(date)

    for row in title_rows:
        tds = row.find_all('td')
        title = tds[1].get_text().replace('Press Enter to show more details.', '')  
        titles.append(title)

    for row in link_rows:
        link = 'https://www.abf.gov.au' + row.a['href']
        links.append(link)
    
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(2)
    
    elem = driver.find_element(By.LINK_TEXT,str(i))
    elem.click()
    time.sleep(1)

In [7]:
print(len(links))

1680


In [37]:
soup = BeautifulSoup(driver.page_source, 'lxml')
    
date_rows = soup.select('div.col-sm-3 span')
title_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-header')})
link_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-content')})
    
for row in date_rows:
    date = row.text
    dates.append(date)

for row in title_rows:
    tds = row.find_all('td')
    title = tds[1].get_text().replace('Press Enter to show more details.', '')  
    titles.append(title)

for row in link_rows:
    link = 'https://www.abf.gov.au' + row.a['href']
    links.append(link)

In [38]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
import io
import urllib.request
import requests


def pdf_to_text(pdf_file):
    text_memory_file = io.StringIO()

    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, text_memory_file, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    # get first 3 pages of the pdf file
    for page in PDFPage.get_pages(pdf_file, pagenos=(0,0)):
        interpreter.process_page(page)
    text = text_memory_file.getvalue()
    text_memory_file.close()
    return text



In [39]:
texts = []
start = 0
finish = len(dates)


# online pdf to text by requests
# response = requests.get("https://www.abf.gov.au/help-and-support-subsite/CustomsNotices/2022-10.pdf")
# pdf_memory_file = io.BytesIO()
# pdf_memory_file.write(response.content)
# print(pdf_to_text(pdf_memory_file))


for i in range(start, finish): # 100개씩 끊어서 돌린 뒤 texts에 추가, 총 1683개의 링크, 17번 돌리기
    url = links[i]
    response = requests.get(url)
    my_raw_data = response.content
    
    # online pdf to text by requests
    response = requests.get(url)
    pdf_memory_file = io.BytesIO()
    pdf_memory_file.write(response.content)
    texts.append(pdf_to_text(pdf_memory_file))


The PDF <_io.BytesIO object at 0x000001F5B8F5C450> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


In [41]:
import pandas as pd

df = pd.DataFrame()
df['date'] = pd.to_datetime(dates)
df['link'] = links
df['title'] = titles
df['text'] = texts

df

,date,link,title,text
0,2022-07-28,https://www.abf.gov.au/help-and-support-subsit...,Indexation of customs duty rates on excise-equ...,Australian Customs Notice \nNo. 2022/35 \nInd...
1,2022-07-22,https://www.abf.gov.au/help-and-support-subsit...,Removal of customs duty on certain electric ve...,Australian Customs Notice \nNo. 2022/34 \nRem...
2,2022-07-21,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,Australian Customs Notice No. 2022/33 \n\nAppl...
3,2022-07-12,https://www.abf.gov.au/help-and-support-subsit...,Temporary duty reduction for goods from Ukraine,Australian Customs Notice \nNo. 2022/32 \nTem...
4,2022-07-11,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,Australian Customs Notice No. 2022/31 \n\nAppl...
...,...,...,...,...
1679,2018-12-10,https://www.abf.gov.au/help-and-support-subsit...,Customs Act 1901 - Part XVB Review of Anti-Dum...,Australian Customs Notice 1996 05 2\n\nCUSTOMS...
1680,2018-12-10,https://www.abf.gov.au/help-and-support-subsit...,Origin of goods packaged within the preference...,Australian Customs Notice 1996 04\n\nORIGIN OF...
1681,2018-12-13,https://www.abf.gov.au/help-and-support-subsit...,Customs Tariff Act 1995,Australian Customs Notice 1996 03\n\nCUSTOMS T...
1682,2018-12-13,https://www.abf.gov.au/help-and-support-subsit...,Amendments to the Customs (Prohibited Imports)...,Australian Customs Notice 1996 02\n\nAMENDMENT...


In [42]:
len(df)

1684

In [44]:
df=df[::-1]
df.head()

,date,link,title,text
1683,2018-12-13,https://www.abf.gov.au/help-and-support-subsit...,Trademarks Act 1995,Australian Customs Notice 1996 01\n\nTRADE MAR...
1682,2018-12-13,https://www.abf.gov.au/help-and-support-subsit...,Amendments to the Customs (Prohibited Imports)...,Australian Customs Notice 1996 02\n\nAMENDMENT...
1681,2018-12-13,https://www.abf.gov.au/help-and-support-subsit...,Customs Tariff Act 1995,Australian Customs Notice 1996 03\n\nCUSTOMS T...
1680,2018-12-10,https://www.abf.gov.au/help-and-support-subsit...,Origin of goods packaged within the preference...,Australian Customs Notice 1996 04\n\nORIGIN OF...
1679,2018-12-10,https://www.abf.gov.au/help-and-support-subsit...,Customs Act 1901 - Part XVB Review of Anti-Dum...,Australian Customs Notice 1996 05 2\n\nCUSTOMS...


In [45]:
df.to_csv('호주_세관_전체.csv', mode='w', encoding='utf-8-sig', index = False)